In [7]:
!pip install -r requirements.txt
import requests
import time
from bs4 import BeautifulSoup
import requests
import csv


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
base_url = 'https://visitseattle.org/events/page/{}'
all_events = []

for page_number in range(1, 39):  # Iterate over pages 1 to 38
    url = base_url.format(page_number)
    res = requests.get(url)
    
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'html.parser')
        selector = "div.search-result-preview > div > h3 > a"
        a_eles = soup.select(selector)
        
        # Extract href attributes from the anchor elements and append to the list
        all_events.extend([x['href'] for x in a_eles])

# Print the list of event URLs
print("event summary:", len(all_events))
print(all_events)

event summary: 341
['https://visitseattle.org/events/dead-bars/', 'https://visitseattle.org/events/emotional-a-night-for-the-heartbroken/', 'https://visitseattle.org/events/loonieboys/', 'https://visitseattle.org/events/corteo/', 'https://visitseattle.org/events/bewitcher/', 'https://visitseattle.org/events/burn-burn-burn/', 'https://visitseattle.org/events/carly-ann-calbero/', 'https://visitseattle.org/events/el-primer-instinto/', 'https://visitseattle.org/events/gary-janetti/', 'https://visitseattle.org/events/jammah/', 'https://visitseattle.org/events/saturday-family-concert-the-harmonica-pocket/', 'https://visitseattle.org/events/seattle-thunderbirds-vs-tri-city-americans-2/', 'https://visitseattle.org/events/hokusai-inspiration-and-influence-from-the-collection-of-the-museum-of-fine-arts-boston/', 'https://visitseattle.org/events/an-evening-with-chris-botti/', 'https://visitseattle.org/events/blueys-big-play/', 'https://visitseattle.org/events/mike-birbiglia-please-stop-the-ride/'

In [7]:
def extract_event_details(detail_url):
    detail_res = requests.get(detail_url)

    if detail_res.status_code == 200:
        detail_soup = BeautifulSoup(detail_res.text, 'html.parser')
        # Extracting details (modify selectors as needed)
        name = detail_soup.select_one('div.medium-6.columns.event-top > h1').text.strip()
        date_time = detail_soup.select_one('div.medium-6.columns.event-top > h4 > span:nth-child(1)').text.strip()
        location = detail_soup.select_one('div.medium-6.columns.event-top > h4 > span:nth-child(2)').text.strip()
        event_type = detail_soup.select_one('div.medium-6.columns.event-top > a:nth-child(3)').text.strip()
        region = detail_soup.select_one('div.medium-6.columns.event-top > a:nth-child(4)').text.strip()
        
        return [name, date_time, location, event_type, region]
    else:
        return None

In [13]:
all_event_details = []
for event_url in all_events:
    event_details = extract_event_details(event_url)
    
    if event_details:
        all_event_details.append(event_details)
print(all_event_details)

[['Dead Bars', '1/19/2024', "Slim's Last Chance", 'Music', 'South Seattle'], ['Emotional: A Night For The Heartbroken', '1/19/2024', 'Chop Suey', 'Music', 'Capitol Hill / Central District'], ['Loonieboys', '1/19/2024', 'Funhouse', 'Music', 'Capitol Hill / Central District'], ['Corteo', 'Now through 1/20/2024', 'Climate Pledge Arena', 'Theatre', 'Queen Anne / Seattle Center'], ['Bewitcher', '1/20/2024', 'Funhouse', 'Music', 'Capitol Hill / Central District'], ['burn burn burn', '1/20/2024', "Slim's Last Chance", 'Music', 'South Seattle'], ['Carly Ann Calbero', '1/20/2024', 'The Royal Room', 'Music', 'South Seattle'], ['El Primer Instinto', '1/20/2024', 'El Corazon', 'Music', 'Downtown'], ['Gary Janetti', '1/20/2024', 'The Neptune Theatre', 'Theatre', 'University District'], ['Jammah', '1/20/2024', 'Sea Monster Lounge', 'Music', 'Wallingford / Greenlake'], ['Saturday Family Concert: The Harmonica Pocket', '1/20/2024', 'Town Hall Seattle', 'Music', 'Downtown'], ['Seattle Thunderbirds vs. 

In [9]:
import csv
csv_header = ['Name', 'Date + Time', 'Location', 'Type', 'Region']
with open('events.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    # Write header
    csv_writer.writerow(csv_header)
    
    # Write event details
    csv_writer.writerows(all_event_details)

print("Event details have been stored in events.csv")

Event details have been stored in events.csv


# Weather

In [7]:
# location name to lat lon
base_url = "https://nominatim.openstreetmap.org/search.php"
# Option 2: use dictionary to represent query params
#           use requests.get(url, params=query_params) to attach the query param dict
query_params = {
    "q": "Wallingford, Seattle",
    "format": "jsonv2"
}

res = requests.get(base_url, params=query_params)
res.json()

[{'place_id': 312844859,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
  'osm_type': 'node',
  'osm_id': 150973716,
  'lat': '47.6594631',
  'lon': '-122.3343417',
  'category': 'place',
  'type': 'suburb',
  'place_rank': 19,
  'importance': 0.3131791431958073,
  'addresstype': 'suburb',
  'name': 'Wallingford',
  'display_name': 'Wallingford, Seattle, King County, Washington, 98015, United States',
  'boundingbox': ['47.6394631', '47.6794631', '-122.3543417', '-122.3143417']}]

In [6]:
base_url = "https://nominatim.openstreetmap.org/search.php"

# Function to get latitude and longitude for a given location and region
def get_lat_long(location, region):
    query_params = {
        "q": f"{location}, {region}",
        "format": "jsonv2"
    }
    res = requests.get(base_url, params=query_params)
    data = res.json()

    if not data:
        query_params_location = {
            "q": f"{location}",
            "format": "jsonv2"
        }
        res_location = requests.get(base_url, params=query_params_location)
        data_location = res_location.json()

        if data_location:
            return data_location[0]["lat"], data_location[0]["lon"]

        query_params_region = {
            "q": f"{region}",
            "format": "jsonv2"
        }
        res_region = requests.get(base_url, params=query_params_region)
        data_region = res_region.json()

        if data_region:
            return data_region[0]["lat"], data_region[0]["lon"]

        return None, None

    return data[0]["lat"], data[0]["lon"]

# Read events from CSV
with open('events.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    header = next(reader)  # Get header
    header += ['Latitude', 'Longitude']  # Add Latitude and Longitude to header

    rows = []
    for row in reader:
        name, date_time, location, event_type, region = row
        latitude, longitude = get_lat_long(location, region)
        rows.append(row + [latitude, longitude])
        time.sleep(1)

# Write to a new CSV file
with open('events_with_coordinates.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(rows)

In [3]:
url = "https://api.weather.gov/points/39.7456,-97.0892"
res = requests.get(url)
res

<Response [200]>

In [ ]:
import csv
import requests

# Function to get weather forecast for a given latitude and longitude
def get_weather_forecast(latitude, longitude):
    url = f"https://api.weather.gov/points/{latitude},{longitude}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        forecast_url = data['properties']['forecast']
        
        # Fetch the forecast data
        forecast_response = requests.get(forecast_url)
        if forecast_response.status_code == 200:
            forecast_data = forecast_response.json()
            return forecast_data['properties']['periods'][0]['detailedForecast']
    
    return None

# Read events from CSV
events = []
with open('events_with_coordinates.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        event = {
            'Name': row['Name'],
            'Date': row['Date + Time'],
            'Location': row['Location'],
            'Type': row['Type'],
            'Region': row['Region'],
            'Latitude': float(row['Latitude']),
            'Longitude': float(row['Longitude'])
        }
        events.append(event)

# Fetch weather forecast for each event
for event in events:
    latitude = event['Latitude']
    longitude = event['Longitude']
    forecast = get_weather_forecast(latitude, longitude)
    
    if forecast:
        event['Forecast'] = forecast
    else:
        event['Forecast'] = 'Forecast not available'

# Write combined information to a new CSV file
fields = ['Name', 'Date', 'Location', 'Type', 'Region', 'Latitude', 'Longitude', 'Forecast']
with open('events_with_forecast.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fields)
    
    # Write header
    writer.writeheader()
    
    # Write event details and forecast
    for event in events:
        writer.writerow(event)

print("Process completed. Check 'events_with_forecast.csv' for the combined information.")